In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error



In [10]:
# Load datasets

temp_df = pd.read_csv("../train.csv", nrows=0)  # Read only the header
total_columns = len(temp_df.columns)
columns_to_use = temp_df.columns[2:total_columns] 
train_data = pd.read_csv("../train.csv", usecols=columns_to_use)


train_data = train_data.drop(columns=[ 'site_id', 'sub_primaryspaceusage', 'season'])

test_data = pd.read_csv("../test.csv", usecols=columns_to_use)

test_data = test_data.drop(columns=[ 'site_id', 'sub_primaryspaceusage', 'season'])
train_data.head()


,building_id,meter,date,meter_reading,building_id_kaggle,site_id_kaggle,sqm,sqft,timezone,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
0,Bear_education_Alfredo,electricity,2016-01-01,2.9050,636.0,4.0,609.8,6564.0,US/Pacific,5.246861,1.927009,0.254484,0.351088,10.801125,1018.888301,172.924863,3.807399
1,Bear_education_Alfredo,electricity,2016-01-02,2.7700,636.0,4.0,609.8,6564.0,US/Pacific,5.993973,1.997893,0.892188,0.409453,11.105558,1014.347411,181.359441,4.202455
2,Bear_education_Alfredo,electricity,2016-01-03,2.6725,636.0,4.0,609.8,6564.0,US/Pacific,5.660314,1.946017,0.778475,0.552568,11.167389,1010.396019,208.978674,4.015919
3,Bear_education_Alfredo,electricity,2016-01-04,4.5650,636.0,4.0,609.8,6564.0,US/Pacific,5.048507,1.987616,-0.268905,0.479493,11.089874,1008.903334,211.377040,3.909701
4,Bear_education_Alfredo,electricity,2016-01-05,4.7825,636.0,4.0,609.8,6564.0,US/Pacific,4.745567,2.007311,0.321921,1.033857,11.723586,1012.747700,170.002007,3.528571


In [11]:
# Preprocessing
def preprocess_data(data):
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    categorical_features = ['meter']
    numerical_features = ['sqm', 'airTemperature', 'cloudCoverage', 'dewTemperature', 
                          'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 
                          'windDirection', 'windSpeed']
    
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    numerical_transformer = StandardScaler()
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    return preprocessor.fit_transform(data), data['meter_reading']  # Return processed features and target

X_train, y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

In [58]:
meter_types = train_data['meter'].unique()  # Get unique meter types from the dataset

# Create a dictionary to hold the models and their history
models = {}
histories = {}
evaluations = {}
predictions = {}
rmses = {}


for meter_type in meter_types:
    # Segment the datasets by meter type
    train_data_meter = train_data[train_data['meter'] == meter_type]
    test_data_meter = test_data[test_data['meter'] == meter_type]
    
    # Preprocess the data
    X_train_meter, y_train_meter = preprocess_data(train_data_meter)
    X_test_meter, y_test_meter = preprocess_data(test_data_meter)
    
    # Create the model for this meter type
    input_shape = X_train_meter.shape[1]  # Get input shape from the preprocessed data
    model = create_model(input_shape)
    
    # Train the model
    print(f'Training model for {meter_type}...')
    history = model.fit(X_train_meter, y_train_meter, validation_split=0.2, epochs=50, batch_size=32)
    
    # Save the model and its history
    models[meter_type] = model
    histories[meter_type] = history
    
    # Evaluate the model
    print(f'Evaluating model for {meter_type}...')
    loss = model.evaluate(X_test_meter, y_test_meter)
    evaluations[meter_type] = loss
    
    # Generate predictions
    print(f'Predicting with model for {meter_type}...')
    prediction = model.predict(X_test_meter).flatten()  # Ensure predictions match the shape of y_test_meter
    
    # Handling NaNs in predictions or test labels, if needed
    if np.isnan(prediction).any() or np.isnan(y_test_meter).any():
        prediction = np.nan_to_num(prediction, nan=0.0)  # Replace NaNs in predictions
        y_test_meter = np.nan_to_num(y_test_meter, nan=0.0)  # Replace NaNs in test labels

    # Calculate RMSE
    rmse = mean_squared_error(y_test_meter, prediction, squared=False)
    rmses[meter_type] = rmse
    print(f'RMSE for {meter_type}: {rmse}\n')
    
    # Store the predictions along with building_id and meter
    results_df = pd.DataFrame({
        'BuildingID': test_data_meter['building_id'].values,
        'Meter': meter_type,
        'ActualMeterReading': y_test_meter,
        'PredictedMeterReading': prediction
    })
    
    # Optionally save to CSV
    results_df.to_csv(f'predictions_{meter_type}.csv', index=False)
    
    # Add to the dictionary if you want to keep in memory
    predictions[meter_type] = results_df
    
    print(f'Completed {meter_type}.\n')  # Separator for readability


/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = data['date'].dt.year
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Training model for electricity...
Epoch 1/50
5527/5527 [==============================] - 2s 352us/step - loss: 45201220.0000 - val_loss: 13972922.0000
Epoch 2/50
5527/5527 [==============================] - 2s 345us/step - loss: 39838044.0000 - val_loss: 13867711.0000
Epoch 3/50
5527/5527 [==============================] - 2s 337us/step - loss: 39733280.0000 - val_loss: 14180063.0000
Epoch 4/50
5527/5527 [==============================] - 2s 334us/step - loss: 39516620.0000 - val_loss: 14548565.0000
Epoch 5/50
5527/5527 [==============================] - 2s 336us/step - loss: 39228392.0000 - val_loss: 14753831.0000
Epoch 6/50
5527/5527 [==============================] - 2s 337us/step - loss: 38967468.0000 - val_loss: 15573729.0000
Epoch 7/50
5527/5527 [==============================] - 2s 339us/step - loss: 38719428.0000 - val_loss: 15736766.0000
Epoch 8/50
5527/5527 [==============================] - 2s 343us/step - loss: 38499644.0000 - val_loss: 16578233.0000
Epoch 9/50
5527/5527 [

/Users/sameeraboppana/work_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE for electricity: 5643.688955366916



/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = data['date'].dt.year
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

...

/Users/sameeraboppana/work_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Training model for steam...
Epoch 1/50
1346/1346 [==============================] - 1s 437us/step - loss: 287742048.0000 - val_loss: 93102031241216.0000
Epoch 2/50
1346/1346 [==============================] - 0s 362us/step - loss: 247161792.0000 - val_loss: 93091360931840.0000
Epoch 3/50
1346/1346 [==============================] - 0s 364us/step - loss: 240491648.0000 - val_loss: 93087896436736.0000
Epoch 4/50
1346/1346 [==============================] - 0s 355us/step - loss: 238238880.0000 - val_loss: 93084650045440.0000
Epoch 5/50
1346/1346 [==============================] - 1s 381us/step - loss: 237147632.0000 - val_loss: 93084817817600.0000
Epoch 6/50
1346/1346 [==============================] - 1s 469us/step - loss: 236527840.0000 - val_loss: 93083962179584.0000
Epoch 7/50
1346/1346 [==============================] - 1s 379us/step - loss: 236003104.0000 - val_loss: 93084046065664.0000
Epoch 8/50
1346/1346 [==============================] - 0s 348us/step - loss: 235575024.0000 - va

/Users/sameeraboppana/work_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
/var/folders/20/6gd3bgbx1kn0xfsyzbfxk8nm0000gn/T/ipykernel_53854/1659862062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Training model for irrigation...
Epoch 1/50
37/37 [==============================] - 0s 2ms/step - loss: 84014.8828 - val_loss: 3.0955
Epoch 2/50
37/37 [==============================] - 0s 794us/step - loss: 83367.4688 - val_loss: 27.3409
Epoch 3/50
37/37 [==============================] - 0s 701us/step - loss: 81870.9922 - val_loss: 166.9110
Epoch 4/50
37/37 [==============================] - 0s 681us/step - loss: 78895.9766 - val_loss: 705.2130
Epoch 5/50
37/37 [==============================] - 0s 693us/step - loss: 74801.5547 - val_loss: 1984.7762
Epoch 6/50
37/37 [==============================] - 0s 683us/step - loss: 70883.4297 - val_loss: 4139.8540
Epoch 7/50
37/37 [==============================] - 0s 674us/step - loss: 68320.2422 - val_loss: 5971.1074
Epoch 8/50
37/37 [==============================] - 0s 717us/step - loss: 67226.3359 - val_loss: 7100.8970
Epoch 9/50
37/37 [==============================] - 0s 803us/step - loss: 66876.7266 - val_loss: 7196.8779
Epoch 10/50
3

/Users/sameeraboppana/work_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [59]:
rmses

{'electricity': 5643.688955366916,
 'chilledwater': 7523772.530578289,
 'gas': 52345.64513098371,
 'hotwater': 2393173.4056963995,
 'solar': 1506.5089486904917,
 'water': 15573.747483523011,
 'steam': 20007.570771014744,
 'irrigation': 432.687690271091}

### RNN

In [69]:
temp_df = pd.read_csv("../train.csv", nrows=0)  # Read only the header
total_columns = len(temp_df.columns)
columns_to_use = temp_df.columns[2:total_columns] 
train_data = pd.read_csv("../train.csv", usecols=columns_to_use)


train_data = train_data.drop(columns=[ 'site_id', 'sub_primaryspaceusage', 'season'])

test_data = pd.read_csv("../test.csv", usecols=columns_to_use)

test_data = test_data.drop(columns=[ 'site_id', 'sub_primaryspaceusage', 'season'])
train_data.head()


train_data = train_data[train_data['meter'] == 'electricity']
test_data = test_data[test_data['meter'] == 'electricity']

In [70]:
# Define categorical and numerical features from your dataset
categorical_features = ['meter']  # Add other categorical features as needed
numerical_features = ['airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR', 
                      'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']  # Based on your dataset

# Combine all features for processing
all_features = categorical_features + numerical_features
target = 'meter_reading'

# One-hot encode categorical features and normalize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Fit on 2016 data and transform both 2016 and 2017 data
X_2016 = preprocessor.fit_transform(train_data[all_features])
X_2017 = preprocessor.transform(test_data[all_features])

# Extract target values
y_2016 = train_data[target].values
y_2017 = test_data[target].values


In [71]:
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define your time_steps (e.g., 30 days)
time_steps = 30

# Create sequences from the prepared data
X_train, y_train = create_sequences(X_2016, y_2016, time_steps=time_steps)
X_test, y_test = create_sequences(X_2017, y_2017, time_steps=time_steps)


In [72]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)


In [74]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define the model
model = Sequential([
    # SimpleRNN layer: Adjust 'units' as needed, input_shape should be [time_steps, num_features]
    SimpleRNN(units=32, input_shape=(None, X_train.shape[2]), activation='relu'),
    # Output layer: Predict the meter reading, hence one unit
    Dense(1)
])

# Compile the model: Use Mean Squared Error for regression tasks
model.compile(optimizer='adam', loss='mse', metrics=[tf.metrics.RootMeanSquaredError()])

# Display the model's architecture
model.summary()

# Train the model: Adjust epochs, batch_size as necessary
history = model.fit(
    train_ds,
    epochs=20,  # Number of epochs to train for
    validation_data=test_ds,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]  # Stop training when validation loss stops improving
)


Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                1344      
                                                                 
 dense_78 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1377 (5.38 KB)
Trainable params: 1377 (5.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
6908/6908 [==============================] - 13s 2ms/step - loss: 53525588.0000 - root_mean_squared_error: 7316.1182 - val_loss: nan - val_root_mean_squared_error: nan
Epoch 2/20
6908/6908 [==============================] - 12s 2ms/step - loss: 53205456.0000 - root_mean_squared_error: 7294.2070 - val_loss: nan - val_root_mean_squared_error: nan
Epoch 3/20
6908/6908 [==============================] - 12s

In [75]:
predictions_2017 = model.predict(test_ds)

6889/6889 [==============================] - 3s 453us/step


In [83]:
# Extract the dates and building IDs for the days we're predicting
# Assuming the 'date' and 'building_id' for the last day of each sequence are what we're predicting for
# This part needs to be adjusted based on how you've structured your data
predicted_dates = test_data['date'].iloc[time_steps:][::time_steps].reset_index(drop=True)
predicted_building_ids = test_data['building_id'].iloc[time_steps:][::time_steps].reset_index(drop=True)

# Create a DataFrame for the predictions
predictions_df = pd.DataFrame({
    'date': predicted_dates,
    'building_id': predicted_building_ids,
    'predicted_meter_reading': predictions_2017.flatten()  # Flatten in case predictions have an extra dimension
})

# Display the head of the predictions DataFrame to check
predictions_df.head()


ValueError: array length 220430 does not match index length 7348

In [88]:
expected_sequences = len(test_data) - time_steps

if len(predictions_2017) == expected_sequences:
    predicted_dates = test_data['date'].iloc[time_steps:].reset_index(drop=True)
    predicted_building_ids = test_data['building_id'].iloc[time_steps:].reset_index(drop=True)

    predictions_df = pd.DataFrame({
        'date': predicted_dates,
        'building_id': predicted_building_ids,
        'predicted_meter_reading': predictions_2017.flatten()  # Adjust this if not one-to-one
    })
else:
    print("The lengths do not match, check sequence creation and prediction steps.")


In [92]:
actual_meter_readings = test_data.loc[test_data.index.isin(predictions_df.index), 'meter_reading']

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(actual_meter_readings, predictions_df['predicted_meter_reading']))
print("RMSE:", rmse)

ValueError: Found input variables with inconsistent numbers of samples: [95600, 220430]